In [3]:
############ ITALY - KML and KMZ FILES


import geopandas as gpd

gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Italy/Calci_2018/Evoluzione Perimetro Incendio Monte Serra 24 set 2018_00.kml', driver='KML')
print(gdf.crs) 
# Reproject to EPSG:32719 for UTM zone 19S)
gdf = gdf.to_crs(epsg=32719)
print(gdf.crs)

EPSG:4326
EPSG:32719


In [4]:
### CALCI
import geopandas as gpd
import pandas as pd
import re

def extract_iso_timestamp(name):
    m = re.search(r'(\d{4})(\d{2})(\d{2})_(\d{2})(\d{2})', str(name))
    if m:
        return f"{m.group(1)}-{m.group(2)}-{m.group(3)} {m.group(4)}:{m.group(5)}"
    else:
        return None

# Create column "iso_timestamp" as text
gdf['iso_timestamp'] = gdf['Name'].apply(extract_iso_timestamp)



# Order time stamps
gdf = gdf.sort_values('iso_timestamp', ascending=False).reset_index(drop=True)

# Delete attributes
gdf = gdf.drop('description', axis=1)
gdf = gdf.drop('timestamp', axis=1)
gdf = gdf.drop('begin', axis=1)
gdf = gdf.drop('end', axis=1)
gdf = gdf.drop('drawOrder', axis=1)
gdf = gdf.drop('icon', axis=1)
gdf = gdf.drop('altitudeMode', axis=1)
gdf = gdf.drop('tessellate', axis=1)
gdf = gdf.drop('extrude', axis=1)
gdf = gdf.drop('visibility', axis=1)
gdf = gdf.drop('snippet', axis=1)
gdf = gdf.drop('Name', axis=1)

gdf['id'] = gdf.index + 1

gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length   

columns_orden = ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

# Save GeoJSON
gdf.to_file('CalciFire_test8.geojson', driver='GeoJSON')

In [28]:
### MASSAROSA - KMZ FILE
import geopandas as gpd
import zipfile
import os

# Ruta al archivo KMZ
kmz_path = '/home/polivera/Documents/NERO - Database/Italy/Massarosa_2022/Evoluzione Perimetro Incendio Massarosa 2022.kmz'

# Carpeta temporal para extraer KML
temp_dir = '/home/polivera/Documents/NERO - Database/Italy/Massarosa_2022'

import os
import zipfile
import geopandas as gpd
import fiona

os.makedirs(temp_dir, exist_ok=True)

# Extraer todos los archivos del KMZ (que es un ZIP)
with zipfile.ZipFile(kmz_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

# Buscar archivo(s) KML extraído(s)
kml_files = [f for f in os.listdir(temp_dir) if f.lower().endswith('.kml')]

if not kml_files:
    raise FileNotFoundError("No se encontró archivo KML en el KMZ extraído.")

# Leer el primer archivo KML para trabajar
kml_path = os.path.join(temp_dir, kml_files[0])
gdf = gpd.read_file(kml_path, driver='KML')

print(fiona.listlayers(kml_path))




['Punti Bozzano', 'Perimetro Incendio Bozzano']


/home/polivera/miniconda3/envs/geo_data/lib/python3.10/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'doc.kml': 'Punti Bozzano' (default), 'Perimetro Incendio Bozzano'. Specify layer parameter to avoid this warning.
  result = read_func(


In [29]:
### MASSAROSA
# Rename attributes
gdf = gdf.rename(columns={
    'Name': 'iso_timestamp'
})

# Order time stamps
gdf = gdf.sort_values('iso_timestamp', ascending=False).reset_index(drop=True)

# Delete attributes
gdf = gdf.drop('description', axis=1)
gdf = gdf.drop('timestamp', axis=1)
gdf = gdf.drop('begin', axis=1)
gdf = gdf.drop('end', axis=1)
gdf = gdf.drop('drawOrder', axis=1)
gdf = gdf.drop('icon', axis=1)
gdf = gdf.drop('altitudeMode', axis=1)
gdf = gdf.drop('tessellate', axis=1)
gdf = gdf.drop('extrude', axis=1)
gdf = gdf.drop('visibility', axis=1)
gdf = gdf.drop('snippet', axis=1)

gdf['id'] = gdf.index + 1

gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length   

columns_orden = ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

# Save GeoJSON
gdf.to_file('MassarosaFire_test2.geojson', driver='GeoJSON')

KeyError: "['snippet'] not found in axis"